In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import numpy as np
import pickle
import torch
import matplotlib.pyplot as plt 
#from fase.core import heaan

from fase import HEAAN
from fase import HEAAN as he
from typing import List, Callable

from fase import hnrf as hnrf

from fase.hnrf.tree import NeuralTreeMaker
from fase.hnrf import heaan_nrf 
from fase.hnrf.hetree import HomomorphicModel 

#import importlib

from time import time

Using CPU version HEAAN


In [3]:
def decrypt_print(ctx, n=20):
    res1 = decrypt(secretKey, ctx)
    print(res1[:n])
    
def decrypt(secretKey, enc):
    featurized = scheme.decrypt(secretKey, enc)
    arr = np.zeros(n, dtype=np.complex128)
    featurized.__getarr__(arr)
    return arr.real

def encrypt(val):
    ctxt = he.Ciphertext()#logp, logq, n)
    vv = np.zeros(n) # Need to initialize to zero or will cause "unbound"
    vv[:len(val)] = val
    scheme.encrypt(ctxt, he.Double(vv), n, logp, logq)
    del vv
    return ctxt

from torch.utils import data

class TabularDataset(data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, X: np.ndarray, y: np.ndarray):
        'Initialization'
        self.X, self.y = X,y

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.X)

    def __getitem__(self, index):
        'Generates one sample of data'

        # Load data and get label
        X = torch.tensor(self.X[index]).float()
        y = torch.tensor(self.y[index])

        return X, y

class Param():
    def __init__(self, n=None, logn=None, logp=None, logq=None, logQboot=None):
        self.n = n
        self.logn = logn
        self.logp = logp
        self.logq = logq 
        self.logQboot = logQboot
        if self.logn == None:
            self.logn = int(np.log2(n))

# Train a RF model  -- smaller example


In [4]:
import fastai
#print(fastai.__version__)

from fastai.basic_data import DataBunch, DataLoader
#from fastai.tabular.data import TabularDataLoaders

from fastai.tabular.learner import Learner
from fastai.metrics import accuracy

from fase.hnrf.tree import CrossEntropyLabelSmoothing
import torch.nn as nn

matcher를 풀고 train하면 성능이 쉽게 올라가지만 중간에 계산값이 +/- 1을 넘어갈 가능성이 높아질 것 같음.  
근데 matcher 안 풀고는 좀처럼 성능이 올라가지 않음... 

In [20]:
from sklearn.tree import BaseDecisionTree
from fase.hnrf.tree import NeuralRF

model_dir = "/home/hoseung/Work/Kinect_BBS_demo/nbs/"

CAM_LIST= {1: "e",
           2: "e",
           3: "a",
           4: "e",
           5: "e",
           6: "e",
           7: "e",
           8: "a",
           9: "a",
           10:"e",
           11:"e",
           12:"e",
           13:"a",
           14:"e"}


dilatation_factor = 15
polynomial_degree = 21


#for action in np.arange(1,15): 
# 3, 9는 아직 없음 , 12번 다시. 
for action in range(1,13):
    cam = CAM_LIST[action]

    fn_model_out = f"trained_model_{action}_{cam}.pickle"
    fn_data_out = f"BBS_dataset_{action}_{cam}.pickle"

    fn_model = model_dir + fn_model_out
    fn_dat = model_dir + fn_data_out
    
    dataset = pickle.load(open(fn_dat, "rb"))

    X_train = dataset["train_x"]
    y_train = dataset["train_y"]
    X_valid = dataset["valid_x"]
    y_valid = dataset["valid_y"]

    #print("min max of input dataset")
    #print(X_train.min(), X_train.max())
    #print(X_valid.min(), X_valid.max())

    rf_model = pickle.load(open(fn_model, "rb"))

    print("model's depth:", rf_model.max_depth)
    print("model's tree count:", rf_model.n_estimators)


    estimators = rf_model.estimators_

    my_tm_tanh = NeuralTreeMaker(torch.tanh, 
                                use_polynomial=True,
                                dilatation_factor=dilatation_factor, 
                                polynomial_degree=polynomial_degree)

    Nmodel =NeuralRF(estimators, my_tm_tanh)

    # NRF 성능 테스트
    with torch.no_grad():
        neural_pred = Nmodel(torch.tensor(X_valid).float()).argmax(dim=1).numpy()

    print(f"Accuracy of tanh : {(neural_pred == y_valid).mean()}")

model's depth: 7
model's tree count: 20
Accuracy of tanh : 0.3239795918367347
model's depth: 7
model's tree count: 20
Accuracy of tanh : 0.5103989089669281
model's depth: 7
model's tree count: 20
Accuracy of tanh : 0.34350850077279754
model's depth: 7
model's tree count: 20
Accuracy of tanh : 0.480054551653597
model's depth: 7
model's tree count: 20
Accuracy of tanh : 0.4448051948051948
model's depth: 7
model's tree count: 20
Accuracy of tanh : 0.4066461116820829
model's depth: 7
model's tree count: 20
Accuracy of tanh : 0.5973406068871463
model's depth: 7
model's tree count: 20
Accuracy of tanh : 0.4641114982578397
model's depth: 7
model's tree count: 20
Accuracy of tanh : 0.4283480979676915
model's depth: 7
model's tree count: 20
Accuracy of tanh : 0.48547505126452495
model's depth: 7
model's tree count: 20
Accuracy of tanh : 0.4429254955570745
model's depth: 7
model's tree count: 20
Accuracy of tanh : 0.7621326042378674


In [21]:
pred = rf_model.predict(X_train)
print(f"Original accuracy : {(pred == y_train).mean()}")
print(f"Accuracy of tanh : {(neural_pred == y_train).mean()}")
print(f"Match between tanh and original : {(neural_pred == pred).mean()}")


####
bs = 128

train_ds = TabularDataset(X_train, y_train)
valid_ds = TabularDataset(X_valid, y_valid)

train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=bs)
fix_dl = DataLoader(train_ds, batch_size=bs, shuffle=False)

Nmodel.freeze_layer("comparator")
Nmodel.freeze_layer("matcher")

for p in Nmodel.parameters():
    print(p.shape, p.requires_grad)

####
data = DataBunch(train_dl, valid_dl, fix_dl=fix_dl)
#data = TabularDataLoaders.from_dsets(train_dl, valid_dl, bs=64)

criterion = CrossEntropyLabelSmoothing()

Original accuracy : 0.9844497607655502


/tmp/ipykernel_890792/1254981868.py:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  print(f"Accuracy of tanh : {(neural_pred == y_train).mean()}")


AttributeError: 'bool' object has no attribute 'mean'

In [9]:
## Fine-tuned NRF model
pred = rf_model.predict(X_valid)

with torch.no_grad():
    neural_pred = Nmodel(torch.tensor(X_valid).float()).argmax(dim=1).numpy()

print(f"Original accuracy : {(pred == y_valid).mean()}")
print(f"Accuracy : {(neural_pred == y_valid).mean()}")
print(f"Same output : {(neural_pred == pred).mean()}")

RuntimeError: einsum(): operands do not broadcast with remapped shapes [original->remapped]: [5852, 241]->[5852, 1, 1, 241] [240, 92, 20]->[1, 92, 20, 240]

In [386]:
pwd

'/home/hoseung/Work/Kinect_BBS_demo/nbs'

In [7]:
from fase.hnrf.hetree import HNRF
action = 1
cam = CAM_LIST[action]
Nmodel = pickle.load(open(f"Nmodel_{action}_{cam}.pickle", "rb"))

이거를 빨리... 

In [44]:
featurizer = heaan_nrf.HETreeFeaturizer(h_rf.return_comparator(), scheme, parms)

In [36]:
xx.max()

0.9887150053714814

In [40]:
neural_pred = Nmodel(torch.tensor(X_train[:1]).float())
#Nmodel(torch.tensor(X_valid[:1]))

In [41]:
neural_pred

tensor([[   -68.3079,  -4021.2930, -13188.7412,  24814.9121, -10345.2754]],
       grad_fn=<AddBackward0>)

여기서 만든 h_rf와 새로 만든 h_rf의 모양, 값 차이 비교

min = 0, 
max = 1인데.. 왜 -1보다 작아질까...? 

In [23]:
pred

[1354.7771911621098,
 -657.0239105224605,
 -13200.904769897461,
 19254.122360229492,
 -9825.276138305662]